In [1]:
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
import pydotplus 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
iperf1 = pd.read_csv('/Users/xiongzhaoqi/Cambridge/Project/traces1000/iperf.csv')
iperf = iperf1.values.tolist()
memcached1 = pd.read_csv('/Users/xiongzhaoqi/Cambridge/Project/traces1000/memcached.csv')
memcached = memcached1.values.tolist()
ping1 = pd.read_csv('/Users/xiongzhaoqi/Cambridge/Project/traces1000/ping.csv')
ping = ping1.values.tolist()
sparkglm1 = pd.read_csv('/Users/xiongzhaoqi/Cambridge/Project/traces1000/sparkglm.csv')
sparkglm = sparkglm1.values.tolist()
sparkkmeans1 = pd.read_csv('/Users/xiongzhaoqi/Cambridge/Project/traces1000/sparkkmeans.csv')
sparkkmeans = sparkkmeans1.values.tolist()
X = iperf + memcached + ping + sparkglm + sparkkmeans
#X = memcached + ping 
Y = 1000*[0] + 1000*[1]  + 1000*[2] + 1000*[3] + 1000* [4]

In [3]:
X = np.array(X)
Y = np.array(Y)

In [4]:
print(len(X))

5000


In [5]:
indices = np.random.permutation(len(X))

In [6]:
print(indices)

[1990 1884 4875 ... 3862 3473 2197]


In [7]:
Training_X = X[indices[:-400]] 
Training_Y = Y[indices[:-400]]
Testing_X = X[indices[-400:]]
Testing_Y = Y[indices[-400:]]

In [8]:
dt = DecisionTreeClassifier(max_depth = 5)
dt.fit(Training_X, Training_Y)
# dt.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
Train = dt.predict(Training_X )
print(accuracy_score(Training_Y, Train))

0.9704347826086956


In [10]:
Predict_Y = dt.predict(Testing_X)
print(classification_report(Testing_Y, Predict_Y))
print(accuracy_score(Testing_Y, Predict_Y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        92
           1       0.96      1.00      0.98        78
           2       1.00      1.00      1.00        60
           3       0.83      0.99      0.90        86
           4       1.00      0.77      0.87        84

   micro avg       0.95      0.95      0.95       400
   macro avg       0.96      0.95      0.95       400
weighted avg       0.96      0.95      0.95       400

0.95


In [11]:
# cm = confusion_matrix(Testing_Y, Predict_Y)
# plt.matshow(cm,cmap=plt.cm.Greens)
# plt.colorbar()
# for x in range(5):
#     for y in range(5):
#         plt.annotate(cm[y,x],xy=(x,y),horizontalalignment='center',verticalalignment='center')
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.show()

In [12]:
dot_data = export_graphviz(dt, out_file=None,
                                feature_names=['proto','src','dst'],  
                                class_names=['iperf','memcached','ping','sparkglm','sparkkmeans']) 
graph = pydotplus.graph_from_dot_data(dot_data) 
graph.write_pdf("/Users/xiongzhaoqi/Cambridge/Project/machinelearning/tree.pdf")

True

In [13]:
feature_names=['proto','src','dst']
# feature_names= ['iperf','memcached','ping','sparkglm','sparkkmeans']
class_names=['iperf','memcached','ping','sparkglm','sparkkmeans']

def get_lineage(tree, feature_names,file):
     proto = []
     src = []
     dst = []
     left      = tree.tree_.children_left
     right     = tree.tree_.children_right
     threshold = tree.tree_.threshold
     features  = [feature_names[i] for i in tree.tree_.feature]
     value = tree.tree_.value
     le='<='               
     g ='>'
     # get ids of child nodes
     idx = np.argwhere(left == -1)[:,0]     

     def recurse(left, right, child, lineage=None):          
          if lineage is None:
               lineage = [child]
          if child in left:
               parent = np.where(left == child)[0].item()
               split = 'l'
          else:
               parent = np.where(right == child)[0].item()
               split = 'r'
          
          lineage.append((parent, split, threshold[parent], features[parent]))
          if parent == 0:
               lineage.reverse()
               return lineage
          else:
               return recurse(left, right, parent, lineage)
     
     for j,child in enumerate(idx):
        clause=' when '
        
        for node in recurse(left, right, child):
            if len(str(node))<3:
                continue
            i = node
            
            if i[1]=='l':  sign=le 
            else: sign=g
            clause=clause+i[3]+sign+str(i[2])+' and '
        
        a = list(value[node][0])
        ind = a.index(max(a)) 
        clause= clause[:-4]+' then '+ str(ind)
        file.write(clause)
        file.write(";\n")
        

In [14]:
threshold = dt.tree_.threshold
features  = [feature_names[i] for i in dt.tree_.feature]

In [15]:
print(threshold)
print(features )

[ 1.10000e+01 -2.00000e+00  6.03900e+03  2.51150e+03  4.99300e+04
 -2.00000e+00 -2.00000e+00 -2.00000e+00  4.31745e+04  2.21575e+04
  4.07400e+04 -2.00000e+00 -2.00000e+00  2.21575e+04 -2.00000e+00
 -2.00000e+00 -2.00000e+00]
['src', 'src', 'dst', 'dst', 'src', 'src', 'src', 'src', 'src', 'dst', 'src', 'src', 'src', 'src', 'src', 'src', 'src']


In [16]:
proto =[]
src =[]
dst =[]
for i,fe in enumerate(features):
    
    if fe == 'proto':
        proto.append(threshold[i])
    elif fe== 'src':
        if threshold[i] != -2.0:
            src.append(threshold[i])
    else:
        dst.append(threshold[i])
      

In [17]:
proto = [int(i) for i in proto]
src = [int(i) for i in src]
dst = [int(i) for i in dst]
proto.sort()
src.sort()
dst.sort()


In [18]:
tree = open("/Users/xiongzhaoqi/Cambridge/Project/machinelearning/tree.txt","w+")
tree.write("proto = ")
tree.write(str(proto))
tree.write(";\n")
tree.write("src = ")
tree.write(str(src))
tree.write(";\n")
tree.write("dst = ")
tree.write(str(dst))
tree.write(";\n")
get_lineage(dt,feature_names,tree)
